# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [3]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("AZURE_OPENAI_API_KEY")
_set_env("AZURE_OPENAI_ENDPOINT")
_set_env("AZURE_OPENAI_API_VERSION")
_set_env("AZURE_OPENAI_DEPLOYMENT_NAME")

In [4]:
llm_config: dict = {
    "config_list": [
        {
            "model": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            "api_type": "azure",
            "api_key": os.environ["AZURE_OPENAI_API_KEY"],
            "base_url": os.environ["AZURE_OPENAI_ENDPOINT"],
            "api_version": os.environ["AZURE_OPENAI_API_VERSION"],
        }
    ]
}

## Define an AutoGen agent

In [5]:
from autogen import ConversableAgent

agent: ConversableAgent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [6]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

Why don't skeletons fight each other?

They don't have the guts!


In [7]:
reply = agent.generate_reply(messages=[{"content": "Repeat the joke.", "role": "user"}])
print(reply)

Of course! Please provide the joke you'd like me to repeat.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [8]:
cathy: ConversableAgent = ConversableAgent(
    name="cathy",
    system_message="Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe: ConversableAgent = ConversableAgent(
    name="joe",
    system_message="Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [9]:
from autogen.agentchat.chat import ChatResult

result: ChatResult = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let’s roll! You know, I tried doing yoga the other day. They say it helps with flexibility, but honestly, the only thing I’m bending is the truth about how I’ll ever get into child’s pose without screaming like a toddler who’s just been told they can’t have dessert! What about you, Joe? Any yoga stories, or are you more of a "let's order takeout and call it a day" kind of person?

--------------------------------------------------------------------------------
joe (to cathy):

Well, Cathy, I definitely lean towards the "order takeout" side of things. I mean, I tried yoga once—it involved me trying to bend into a position called "downward dog," but honestly, I looked more like a confused cat that just realized it forgot where it buried its favorite toy! But hey, at least when I’m eating takeout, I don’t have t

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [10]:
import pprint

pprint.pprint(result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Hey Joe! Alright, let’s roll! You know, I tried doing yoga the '
             'other day. They say it helps with flexibility, but honestly, the '
             'only thing I’m bending is the truth about how I’ll ever get into '
             'child’s pose without screaming like a toddler who’s just been '
             'told they can’t have dessert! What about you, Joe? Any yoga '
             'stories, or are you more of a "let\'s order takeout and call it '
             'a day" kind of person?',
  'name': 'cathy',
  'role': 'user'},
 {'content': 'Well, Cathy, I definitely lean towards the "order takeout" side '
             'of things. I mean, I tried yoga once—it involved me trying to '
             'bend into a position called "downward dog," but honestly, I '
             'looked more like a confused cat that just realized it forgot '
             'where it buried its 

In [11]:
pprint.pprint(result.cost)

{'usage_excluding_cached_inference': {'total_cost': 0},
 'usage_including_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 297,
                                                                 'cost': 0.00024239999999999998,
                                                                 'prompt_tokens': 428,
                                                                 'total_tokens': 725},
                                      'total_cost': 0.00024239999999999998}}


In [12]:
pprint.pprint(result.summary)

('Cathy: Oh, Joe, I can totally relate! "Downward dog" sounds like a bad '
 'Tinder date gone wrong! And waiting for takeout is its own kind of '
 'workout—like a waiting game with the universe. You’re just praying they '
 'don’t mix up your order and send you a salad instead of that cheesy pizza '
 'you’ve been dreaming about. It’s like, “What am I supposed to do with this '
 'kale?! Am I supposed to dip it in ranch and pretend it’s a fry?”')


## Get a better summary of the conversation

In [13]:
result: ChatResult = joe.initiate_chat(
    cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let’s roll! You know, I tried doing yoga the other day. They say it helps with flexibility, but honestly, the only thing I’m bending is the truth about how I’ll ever get into child’s pose without screaming like a toddler who’s just been told they can’t have dessert! What about you, Joe? Any yoga stories, or are you more of a "let's order takeout and call it a day" kind of person?

--------------------------------------------------------------------------------
joe (to cathy):

Well, Cathy, I definitely lean towards the "order takeout" side of things. I mean, I tried yoga once—it involved me trying to bend into a position called "downward dog," but honestly, I looked more like a confused cat that just realized it forgot where it buried its favorite toy! But hey, at least when I’m eating takeout, I don’t have t

In [14]:
pprint.pprint(result.summary)

('Joe and Cathy humorously discuss their aversion to yoga, sharing relatable '
 'experiences and comparisons between trying yoga poses and their preference '
 'for ordering takeout. They both find the wait for food to be a form of '
 'patience stretching while joking about the potential for disappointing food '
 'orders.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [15]:
cathy: ConversableAgent = ConversableAgent(
    name="cathy",
    system_message="Your name is Cathy and you are a stand-up comedian. "
    "After a few turns, end the conversation: when you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe: ConversableAgent = ConversableAgent(
    name="joe",
    system_message="Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"]
    or "Goodbye" in msg["content"],
)

In [16]:
chat_result = joe.initiate_chat(
    recipient=cathy, message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, here’s one for you: Why did the scarecrow win an award? Because he was outstanding in his field! What do you think? Got any jokes up your sleeve?

--------------------------------------------------------------------------------
joe (to cathy):

Oh, that's a classic! I love a good pun. Here’s one for you: Why don’t scientists trust atoms? Because they make up everything! How's that for a little humor?

--------------------------------------------------------------------------------
cathy (to joe):

Cathy: That's a good one, Joe! Atoms really know how to build up a conversation, don't they? Here’s another: I told my computer I needed a break, and now it won’t stop sending me beach wallpapers! Got any more up your sleeve?

--------------------------------------------------------------------------------
joe (to c

In [17]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was about the librarian and paranoia! She whispered, “They’re right behind you.” It was a great one! You really brought the laughs today. But hey, if you've gotta go, I understand! 

--------------------------------------------------------------------------------
cathy (to joe):

Cathy: Thanks, Joe! I had a blast sharing laughs with you. Let’s do this again sometime! I gotta go!

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (dc5ca2ae-0104-409e-9de9-774af1bbe0e2): Termination message condition on agent 'joe' met
